In [1]:
import pandas as pd
import os
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random

In [2]:
root = os.path.dirname(os.getcwd())
sys.path.append(root)

from src.utils.models import Preprocessor, LSTM_Generator

df = pd.read_csv(root + os.sep + 'data'+ os.sep + 'BASE.csv')

In [3]:
df

,Unnamed: 0,title,quote
0,0,10 things i hate about you,Who knocked up your sister?
1,1,10 things i hate about you,"I was watching you out there, before. I've nev..."
2,2,10 things i hate about you,"You're 18, you don't know what you want. And y..."
3,3,10 things i hate about you,"Ooh, see that, there. Who needs affection when..."
4,4,10 things i hate about you,"Just 'cause you're beautiful, that doesn't mea..."
...,...,...,...
6289,6289,the lord of the rings: the fellowship of the ring,"If by my life or death I can protect you, I wi..."
6290,6290,the lord of the rings: the return of the king,Certainty of death. Small chance of success. W...
6291,6291,the lord of the rings: the return of the king,The journey doesn't end here. Death is just an...
6292,6292,the lord of the rings: the return of the king,I see in your eyes the same fear that would ta...


In [3]:
with open(root + os.sep + 'data'+ os.sep + 'corpus.txt') as f:
    corpus = f.read()

In [6]:
model = LSTM_Generator(corpus[:600000])

Corpus length: 600000
Total chars: 86
Number of sequences: 199987
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               110080    
_________________________________________________________________
dense (Dense)                (None, 86)                11094     
Total params: 121,174
Trainable params: 121,174
Non-trainable params: 0
_________________________________________________________________


In [3]:
p = Preprocessor(df)
p.preprocess()

Corpus length: 630844
Total chars: 52
Number of sequences: 210268


In [5]:
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(cp.maxlen, len(cp.chars))),
        layers.LSTM(128),
        layers.Dense(len(cp.chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [6]:
epochs = 40
batch_size = 128

model.fit(cp.X, cp.Y, batch_size=batch_size, epochs=40)

Epoch 1/40
1643/1643 [==============================] - 82s 49ms/step - loss: 1.9577
Epoch 2/40
1643/1643 [==============================] - 76s 46ms/step - loss: 1.6627
Epoch 3/40
1643/1643 [==============================] - 79s 48ms/step - loss: 1.5851
Epoch 4/40
1643/1643 [==============================] - 83s 50ms/step - loss: 1.5425
Epoch 5/40
1643/1643 [==============================] - 78s 48ms/step - loss: 1.5142
Epoch 6/40
1643/1643 [==============================] - 80s 49ms/step - loss: 1.4930
Epoch 7/40
1643/1643 [==============================] - 81s 49ms/step - loss: 1.4758
Epoch 8/40
1643/1643 [==============================] - 81s 50ms/step - loss: 1.4631
Epoch 9/40
1643/1643 [==============================] - 82s 50ms/step - loss: 1.4523
Epoch 10/40
1643/1643 [==============================] - 84s 51ms/step - loss: 1.4425
Epoch 11/40
1643/1643 [==============================] - 86s 52ms/step - loss: 1.4353
Epoch 12/40
1643/1643 [==============================] - 70s 42

In [16]:
sequence = 'certainty of death. small chance of succ'
cp.generate(model, temperature=0.2, sentence=sequence)

'ession of the president of the started t'

In [9]:
model.save(root + os.sep + 'models/Base_Quote_Generator.h5')

In [ ]:
## Word Base

In [3]:
cp = Preprocessor(df)
cp.preprocess(maxlen=5, option='word')

Corpus length: 630844
Total words: 118223
Unique words before ignoring: 11540
Ignoring words with frequency < 2
Unique words after ignoring: 5173
Number of sequences: 28970


In [4]:
cp.X.shape

(28970, 5, 5173)

In [38]:
word_model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(cp.maxlen, len(cp.tokens))),
        layers.LSTM(128),
        layers.Dense(len(cp.tokens), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
word_model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [39]:
word_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               2714624   
_________________________________________________________________
dense_1 (Dense)              (None, 5173)              667317    
Total params: 3,381,941
Trainable params: 3,381,941
Non-trainable params: 0
_________________________________________________________________


In [40]:
epochs = 100
batch_size = 128

word_model.fit(cp.X, cp.Y, batch_size=batch_size, epochs=epochs)

Epoch 1/100
227/227 [==============================] - 27s 112ms/step - loss: 6.3679
Epoch 2/100
227/227 [==============================] - 25s 110ms/step - loss: 5.7528
Epoch 3/100
227/227 [==============================] - 25s 110ms/step - loss: 5.3173
Epoch 4/100
227/227 [==============================] - 26s 113ms/step - loss: 4.8622
Epoch 5/100
227/227 [==============================] - 24s 106ms/step - loss: 4.4429
Epoch 6/100
227/227 [==============================] - 23s 101ms/step - loss: 4.0802
Epoch 7/100
227/227 [==============================] - 23s 101ms/step - loss: 3.6946
Epoch 8/100
227/227 [==============================] - 23s 101ms/step - loss: 3.3225
Epoch 9/100
227/227 [==============================] - 23s 102ms/step - loss: 2.9886
Epoch 10/100
227/227 [==============================] - 23s 103ms/step - loss: 2.7122
Epoch 11/100
227/227 [==============================] - 24s 107ms/step - loss: 2.4848
Epoch 12/100
227/227 [==============================] - 24s 106

In [41]:
preds = word_model.predict(cp.X[0].reshape(1, cp.X[0].shape[0], cp.X[0].shape[1]))[0]

In [14]:
preds.shape

(5173,)

In [43]:
word_model.save(root + os.sep + 'models' + os.sep + 'Word_Base_LSTM.h5')

In [42]:
generated = ''
sentence = 'certainty of death small chance'
sentence = sentence.split()

for i in range(10):
    x_pred = np.zeros((1, cp.maxlen, len(cp.tokens)))
    for t, char in enumerate(sentence):
        x_pred[0, t, cp.token_indices[char]] = 1.0 #One-Hot Array
    preds = word_model.predict(x_pred, verbose=0)[0]
    next_index = cp.sample(preds, temperature=1)
    next_char = cp.indices_token[next_index]
    sentence = sentence[1:]
    sentence.append(next_char)
    generated += next_char + ' '

generated

'hard moments the i fifty i certain food gone but '

In [29]:
print(sentence)

None


In [36]:
sentence = 'certainty of death small chance'.split()
sentence = sentence[1:]
sentence.append('of')
sentence

['of', 'death', 'small', 'chance', 'of']

In [4]:
model = keras.models.load_model(root + os.sep + 'models'+ os.sep + 'Base_Quote_LSTM.h5')

In [27]:
p.generate(model, quote_len=1000, temperature=0.3)

"e a prosperast. i see the sound of the states the park of the other man was the story. i should be a beat in the fuckin' beautiful the good hall fucking man was a guy. and the brain our or when you go to the path of the country, the started to be the same world. the second in the world, they think this is a good more there was a way the most things with the signs in the rest of the problem with the world. there's no moment of the started the world. i was a little distites of something that i have a world, the said the children. i have a fear and the country are like a bitch in the world. i said the started in the time in the world, and the start of the world. the butt? i would be a little but i see you think i was a brother are because the way the president. i was a secret the more than the park of the dark the country. i see you will be a great community of the troubless of the bast friends of the most month. the day you want to be a little father to the more than a start in the stat

In [45]:
p.generate(model, quote_len=100, temperature=0.3, sentence=sentence)

'ess out of the man will find the world. i love the cheat off the second the second in the last times'

In [30]:
gan_model = keras.models.load_model(root + os.sep + 'models'+ os.sep + 'Type3_Gan.h5')

In [33]:
p.preprocess(mode='gan')

Corpus length: 630844
Total chars: 52
Number of sequences: 210255


In [51]:
p.generate(gan_model, mode='gan')

'pssr ssssen ssse sen apsen apse sauipsen'

In [37]:
sentence = 'certainty of death. small chance of succ'
len(sentence)

40

In [46]:
len('ess out of the man will find the world. i love the cheat off the second the second in the last times')

100

In [47]:
len('e sssawu aufosse sssrawpltssrawu e e apa')

40

In [3]:
from src.utils.models import LSTM_Generator

In [4]:
base = LSTM_Generator(df)

Corpus length: 630844
Total chars: 52
Number of sequences: 210268
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               92672     
_________________________________________________________________
dense (Dense)                (None, 52)                6708      
Total params: 99,380
Trainable params: 99,380
Non-trainable params: 0
_________________________________________________________________


In [5]:
base.predict()

'fw&1}47z7"o!13/0yszp&&3;,jy7x;54vzaf7/bj'